***Proposed Approach***:

The proposed solution employs LangGraph to orchestrate a structured, modular workflow for automating blog generation using open-source Large Language Models (LLMs). Each step in the pipeline sequentially processes input prompts, generating initial content, performing iterative refinement, and applying quality checks. LangSmith tracing is integrated for effective debugging and monitoring, ensuring the generated content meets desired quality standards before the final output is delivered.
![LangGraph Workflow for Automated Blog Generation](output.png)


### **🚀 Steps for Implementing the Blog Generation Workflow**

 **Step 1: Set Up the Environment**
   - Install required libraries: `langchain`, `langgraph`, `transformers`, `huggingface_hub`, `langsmith`.
   - Configure LangSmith for debugging and monitoring.
  
  **!pip install -r requirements.txt**

In [98]:
import langgraph
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFaceHub
from langsmith import trace
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")

**Step2: Integrate Open-Source LLM**
   - Use **Hugging Face models** (e.g., `mistralai/Mistral-7B`, `tiiuae/falcon-7b-instruct`).
   - Implement **Hugging Face API calls** to generate content.

In [103]:
#Initialize the open-source LLM (Hugging face)
llm=ChatOpenAI(model="gpt-4o")

**Step3: Define the Workflow with LangGraph**
   - Create a **graph-based workflow** where each node represents a step in blog generation.
   - Define the following key nodes:
     - **Input Node**: Accepts blog topic or keywords.
     - **Outline Generation Node**: Uses an LLM to create a structured blog outline.
     - **Content Generation Node**: Expands each outline section into a full blog.
     - **Content Refinement Node**: Enhances grammar, coherence, and readability.
     - **Debugging and Quality Check Node**: Uses LangSmith to monitor execution.
     - **Final Blog Output Node**: Produces the structured blog post.

In [104]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, List, Optional
from pydantic import BaseModel
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import tools_condition
from langchain.prompts import PromptTemplate
from langchain_core.tracers import LangChainTracer
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.tracers.langchain import wait_for_all_tracers
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

# Initialize tracer
tracer = LangChainTracer(project_name="blog-generator-project")
callback_manager = CallbackManager([tracer])

class BlogPost(BaseModel):
    title: str
    content: str

class BlogState(BaseModel):
    topic: Optional[str] = None
    outline: Optional[str] = None
    content: Optional[str] = None
    posts: List[BlogPost] = []
    current_post: Optional[BlogPost] = None

# Define state type for type checking
class GraphState(TypedDict):
    blog: BlogState

In [109]:
# Define the nodes
# Define the nodes
def input_topic(state: GraphState) -> GraphState:
    """Get input from user about blog topic"""
    blog_state = state["blog"]
    blog_state.topic = input("Enter the blog topic: ")
    return {"blog": blog_state}

def generate_outline(state: GraphState) -> GraphState:
    """Generate a structured blog outline based on the topic"""
    blog_state = state["blog"]
    
    prompt = PromptTemplate(
        input_variables=["topic"],
        template="Generate a structured blog outline for the topic: {topic}"
    )
    
    # Fix: Handle the response correctly based on what the LLM returns
    response = llm.invoke(prompt.format(topic=blog_state.topic))
    
    # Check if response is a string or an object with content attribute
    if hasattr(response, 'content'):
        blog_state.outline = response.content
    else:
        # If response is a string
        blog_state.outline = str(response)
    
    return {"blog": blog_state}

def generate_content(state: GraphState) -> GraphState:
    """Expand the outline into full blog content"""
    blog_state = state["blog"]
    
    prompt = PromptTemplate(
        input_variables=["outline"],
        template="You are an expert blog writer known for writing engaging, well-structured, and "
            "informative content. Using the provided outline, expand it into a complete blog post with "
            "an engaging introduction, clear section headings, and a compelling conclusion.\n\n"
            "Outline:\n{outline}\n\n"
            "Ensure the blog is detailed, well-structured, and appeals to a professional audience."
    )
    
    # Fix: Handle the response correctly based on what the LLM returns
    response = llm.invoke(prompt.format(outline=blog_state.outline))
    
    # Check if response is a string or an object with content attribute
    if hasattr(response, 'content'):
        blog_state.content = response.content
    else:
        # If response is a string
        blog_state.content = str(response)
    
    # Create a new blog post with the generated content
    title = f"Blog Post about {blog_state.topic}"
    new_post = BlogPost(title=title, content=blog_state.content)
    blog_state.posts.append(new_post)
    blog_state.current_post = new_post
    
    return {"blog": blog_state}

def decide_next(state: GraphState) -> dict:
    """Decision node to determine next steps"""
    print("\nCurrent blog post created:")
    print(f"Title: {state['blog'].current_post.title}")
    print(f"Preview: {state['blog'].current_post.content}")
    
    more_posts = input("Create another blog post? (y/n): ").lower() == 'y'
    if more_posts:
        return {"next": "create_more"}
    else:
        return {"next": "end"}

In [116]:
# Create the graph
from IPython.display import Image, display
def build_blog_graph():
    # Initialize the state graph
    workflow = StateGraph(GraphState)
    
    # Add the nodes
    workflow.add_node("input_topic", input_topic)
    workflow.add_node("generate_outline", generate_outline)
    workflow.add_node("generate_content", generate_content)
    workflow.add_node("decide_next", decide_next)
    
    # Add the edges
    workflow.add_edge(START, "input_topic")
    workflow.add_edge("input_topic", "generate_outline")
    workflow.add_edge("generate_outline", "generate_content")
    workflow.add_edge("generate_content", "decide_next")
    
    
    # Define conditional edges using a router
    workflow.add_conditional_edges(
        "decide_next",
        lambda x: x["next"],
        {
            "create_more": "input_topic",
            "end": END
        }
    )
    
    # Compile the graph
    graph = workflow.compile()
    ## View
    return graph 

In [118]:
if __name__ == "__main__":
    try:
        # Build the graph
        blog_graph = build_blog_graph()
        
        # Initialize with the correct state structure
        initial_state = {"blog": BlogState()}
        
        # Run the graph with LangSmith tracing
        final_state = blog_graph.invoke(
            initial_state,
            {"callbacks": [tracer]}
        )
        
        # Ensure all blog posts are saved
        print("\nFinal Blog Posts:")
        for i, post in enumerate(final_state["blog"].posts):
            file_name = f"blog_post_{i+1}.md"
            with open(file_name, "w", encoding="utf-8") as f:
                f.write(f"# {post.title}\n\n{post.content}")
            print(f"Saved complete blog post to {file_name}")
        
        # Wait for LangSmith traces to complete
        wait_for_all_tracers()
        print("All LangSmith traces uploaded successfully!")

    except Exception as e:
        print(f"An error occurred: {str(e)}")



Current blog post created:
Title: Blog Post about Intro to Machine Learning
Preview: # Introduction to Machine Learning: A Beginner's Guide

## Introduction

Did you know that by 2025, the global machine learning market is expected to ...


Final Blog Posts:
Saved complete blog post to blog_post_1.md
All LangSmith traces uploaded successfully!


# Blog Post about Intro to Machine Learning

# Introduction to Machine Learning: A Beginner's Guide

## Introduction

Did you know that by 2025, the global machine learning market is expected to reach $117 billion? This staggering statistic underscores the profound impact machine learning (ML) has on our everyday lives, from personalized recommendations on Netflix to autonomous vehicles navigating our roads. But what exactly is machine learning, and why is it so crucial in today's technological landscape?

Machine learning is a subset of artificial intelligence (AI) that allows systems to learn and improve from experience without being explicitly programmed. It's the driving force behind many of the innovations we see today, making it an essential area of study and application in the modern world.

In this blog post, we'll delve into the fundamentals of machine learning, explore its various types and components, discuss how it works, and examine its applications across different industries. Whether you're a curious newcomer or someone looking to refresh your knowledge, this guide will provide valuable insights into the fascinating world of machine learning.

## What is Machine Learning?

### Basic Concept

At its core, machine learning is about enabling computers to learn from data and make decisions or predictions. Instead of following static instructions, machines use algorithms to identify patterns and make data-driven decisions.

### Types of Machine Learning

- **Supervised Learning**: In supervised learning, algorithms are trained on labeled data. For example, a model might learn to recognize cats in photos by being shown thousands of labeled images of cats.
  
- **Unsupervised Learning**: This type involves algorithms that work with unlabeled data to find hidden patterns or intrinsic structures. An example is clustering customers based on purchasing behavior without predefined categories.

- **Reinforcement Learning**: Here, algorithms learn by interacting with an environment and receiving feedback in the form of rewards or penalties. It's commonly used in robotics and gaming, such as teaching a robot to walk or an AI to play chess.

## Key Components of Machine Learning

### Data

Data is the lifeblood of machine learning. The quality and quantity of data significantly influence the performance of a model. Machines use data to learn patterns and make predictions.

### Algorithms

Algorithms are the mathematical models that process data and learn from it. They form the backbone of machine learning, enabling systems to transform input data into actionable insights.

### Models

Models are the output of machine learning algorithms applied to data. They are used to make predictions or decisions based on new data. For instance, a weather prediction model forecasts future conditions based on historical data.

## How Machine Learning Works

### Data Collection

The process begins with collecting relevant data from various sources, ensuring it is comprehensive and representative of the task at hand.

### Data Preprocessing

Once collected, data must be cleaned and prepared, which involves handling missing values, normalizing data, and transforming it into a format suitable for training.

### Training the Model

Training involves feeding the prepared data into a machine learning algorithm, allowing it to learn and create a model. This step is crucial as it determines the model's ability to generalize from training to real-world scenarios.

### Evaluation

After training, the model is evaluated using test data to assess its accuracy and performance. This step ensures that the model can make reliable predictions on unseen data.

### Deployment

Once a model is evaluated and refined, it is deployed in real-world applications, where it can operate autonomously or assist human decision-making.

## Applications of Machine Learning

- **Healthcare**: Machine learning is revolutionizing healthcare by enabling predictive diagnostics and personalized treatment plans.

- **Finance**: It's used for fraud detection, risk management, and algorithmic trading, enhancing financial security and efficiency.

- **Retail**: ML powers recommendation systems, helping retailers understand customer preferences and optimize inventory.

- **Transportation**: From autonomous vehicles to route optimization, machine learning is reshaping how we move.

- **Other Industries**: Agriculture, education, manufacturing, and more are leveraging ML for improved outcomes and efficiency.

## Benefits and Challenges of Machine Learning

### Benefits

Machine learning offers numerous benefits, including improved efficiencies, enhanced data-driven insights, and the automation of complex tasks.

### Challenges

Despite its advantages, machine learning faces challenges like data privacy concerns, algorithmic bias, and the need for large, high-quality datasets.

## Getting Started with Machine Learning

### Basic Skills Required

To start with machine learning, one should have a good grasp of programming, a solid understanding of statistics, and familiarity with data analysis.

### Resources for Learning

Numerous online courses, books, and tutorials are available for beginners, such as Coursera's ML courses or the book "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow."

### Tools and Libraries

Popular tools and libraries include Python, TensorFlow, and Scikit-learn, which offer robust frameworks for building and deploying ML models.

## Future of Machine Learning

### Trends

Emerging trends like edge computing and explainable AI are shaping the future of machine learning, promising more efficient and understandable models.

### Potential Impact

Machine learning's evolution will likely lead to significant advancements across industries, from smarter AI systems to improved human-machine interactions.

## Conclusion

In summary, machine learning is a transformative technology with far-reaching implications. By understanding its fundamentals, applications, and future trends, you can appreciate its role in driving innovation and efficiency.

**Call to Action**: Whether you're looking to build a career in this exciting field or simply stay informed, now is the perfect time to start exploring machine learning. Dive into learning resources, engage with communities, and stay updated on industry developments.

## Additional Resources

- **Books**: "Deep Learning" by Ian Goodfellow, "Pattern Recognition and Machine Learning" by Christopher Bishop.
- **Online Communities**: Reddit's Machine Learning community, Kaggle's data science hub.
- **Blogs and Websites**: Towards Data Science, Machine Learning Mastery.

Embark on your machine learning journey today, and be part of shaping the technological landscape of tomorrow.